In [1]:
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE, ExactEigensolver
from qiskit.aqua.operators import Z2Symmetries
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.chemistry import FermionicOperator
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.chemistry.components.initial_states import HartreeFock

### Step 1: Define a molecule
Using the sto3g basis with the PySCF driver as an example, we can describe a molecule.

In [2]:
# LiH molecule with interatomic distance of 1.6 Angstrom
driver = PySCFDriver(atom='Li .0 .0 .0; H .0 .0 1.6', unit=UnitsType.ANGSTROM,
                     charge=0, spin=0, basis='sto3g')
molecule = driver.run()

### Step 2: Prepare qubit Hamiltonian
Here, we setup the molecular orbitals to be considered and can reduce the problem size when we map to the qubit Hamiltonian. 

We are also able to define the mapping type for the qubit Hamiltonian, which - in some cases - allows one to further reduce the problem size.

In [11]:
# depending on what I choose, the number of required q-bits changes
#map_type = 'parity'
#map_type = 'jordan_wigner'
map_type = 'bksf'

In [12]:
# please be aware that the idx here with respective to original idx
freeze_list = [0]
remove_list = [-3, -2] # negative number denotes the reverse order

In [13]:
h1 = molecule.one_body_integrals
h2 = molecule.two_body_integrals
nuclear_repulsion_energy = molecule.nuclear_repulsion_energy

num_particles = molecule.num_alpha + molecule.num_beta
num_spin_orbitals = molecule.num_orbitals * 2
print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
print("# of electrons: {}".format(num_particles))
print("# of spin orbitals: {}".format(num_spin_orbitals))

HF energy: -8.854072040283649
# of electrons: 4
# of spin orbitals: 12


In [14]:
# prepare full idx of freeze_list and remove_list
# convert all negative idx to positive
remove_list = [x % molecule.num_orbitals for x in remove_list]
freeze_list = [x % molecule.num_orbitals for x in freeze_list]
# update the idx in remove_list of the idx after frozen, since the idx of orbitals are changed after freezing
remove_list = [x - len(freeze_list) for x in remove_list]
remove_list += [x + molecule.num_orbitals - len(freeze_list)  for x in remove_list]
freeze_list += [x + molecule.num_orbitals for x in freeze_list]

# prepare fermionic hamiltonian with orbital freezing and eliminating, and then map to qubit hamiltonian
# and if PARITY mapping is selected, reduction qubits
energy_shift = 0.0
qubit_reduction = True if map_type == 'parity' else False

ferOp = FermionicOperator(h1=h1, h2=h2)
if len(freeze_list) > 0:
    ferOp, energy_shift = ferOp.fermion_mode_freezing(freeze_list)
    num_spin_orbitals -= len(freeze_list)
    num_particles -= len(freeze_list)
if len(remove_list) > 0:
    ferOp = ferOp.fermion_mode_elimination(remove_list)
    num_spin_orbitals -= len(remove_list)

# ferOp = fermionic operator    
qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
qubitOp = Z2Symmetries.two_qubit_reduction(qubitOp, num_particles) if qubit_reduction else qubitOp
qubitOp.chop(10**-10)

print(qubitOp)
print(qubitOp.print_details())

Representation: paulis, qubits: 15, size: 226
IIIIIIIIIIIIIII	(-0.207659335019706+0j)
IIIIIIIIIIZZZZZ	(-0.04001650910163407+0j)
IIIIIIZZZZIIIIZ	(-0.12914560136887496+0j)
IIIIIIZZZZZZZZI	(0.05263651528550096+0j)
IIIZZZIIIYIIIIZ	(-0.015008967542955073+0j)
IIIZZZIIIYZZZZI	(-0.004191566320146923+0j)
IIIIIIZZZYIIIZI	(0.015008967542955073+0j)
IIIIIIZZZYZZZIZ	(0.004191566320146923+0j)
IIIZZZIIIZIIIYZ	(0.006707552889272897+0j)
IIIZZZZZZIIIIYI	(-0.005456156289319231+0j)
IIIIIIIIIIZZZYI	(-0.006707552889272897+0j)
IIIIIIZZZZZZZYZ	(0.005456156289319231+0j)
IIIZZZIIIZIIIZI	(-0.30546618417220484+0j)
IIIZZZIIIZZZZIZ	(0.0824794937822228+0j)
IIIIIIZZZZIIIIY	(-0.00031512637669251116+0j)
IIIZZZZZZIIIIZY	(-0.0028624550781848565+0j)
IIIIIIIIIIZZZZY	(0.00031512637669251116+0j)
IIIZZZIIIZZZZIY	(0.0028624550781848565+0j)
IZZIIZIIZIIIZII	(-0.04001650910163407+0j)
IZZIIZIIZIZZIZZ	(0.12182774215820463+0j)
ZIYIIZIIZIIIZII	(-0.00031512637669251246+0j)
ZIYIIZIIZIZZIZZ	(-0.012144897228085896+0j)
IZYIZIIZIIIZIII	(0.0